**E2**

In [153]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing
from sklearn import metrics

loc='/content/drive/MyDrive/Other-Material/'

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [159]:
def pred_fn(X,w):
  x=X.dot(w)
  return 1/(1+np.exp(-x))
def  cal_cost(x,w,y):
    m = len(y)
    pred=pred_fn(x,w)
    cost = -np.sum(y.T.dot(np.log(pred))+(np.subtract(1,y)).T.dot(np.log(np.subtract(1,pred))))/m
    return cost
def gradient_descent(X,y,theta,learning_rate=0.001,iterations=100,p=0.005):
    m = len(y)
    j_hist=[]
    prev=-1
    cost=0
    for it in range(iterations):
        prediction = pred_fn(X,theta)
        theta = theta +(1/m)*learning_rate*( X.T.dot((y - prediction)))
        cost = cal_cost(X,theta,y)
        #cost2 = metrics.log_loss(y,prediction)
        #print(cost,cost2)
        j_hist.append(cost)
        if abs(cost-prev)<p:
          break
        prev=cost  
    return j_hist,theta
    
alpha=[0.1,0.05,0.01,0.05,0.001]
rho=[0.5,0.2,0.05,0.01,0.005]
epoch=[100,1000,2000,3000,5000]

alpha=[0.05]
epoch=[10000]
a=0


In [160]:
#Kfold https://www.askpython.com/python/examples/k-fold-cross-validation
from sklearn.model_selection import KFold
df = df.sample(frac = 1)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X=(X-X.mean())/X.std()
X.insert(0,"w0",np.ones(X.shape[0]))

k = 5
kf = KFold(n_splits=k, random_state=None)
accuracy_fold = np.empty((k,1), dtype=float)
class_precision = np.empty((k,3), dtype=float)
class_recall = np.empty((k,3), dtype=float)

fold=0
for train_index , test_index in kf.split(X):
    print(fold,"th fold...")
    #print(train_index,test_index)
    X_train , X_test = X.iloc[train_index,:].to_numpy(),X.iloc[test_index,:].to_numpy()
    Y_train , Y_test = Y[train_index].to_numpy() , Y[test_index].to_numpy()
    Y_train=Y_train.reshape(len(Y_train),1)
    Y_test=Y_train.reshape(len(Y_train),1)

    predictions=[]
    w_ori= np.random.randn(X_train.shape[1],1)
    for i in range(3):
      Y_train_i=Y_train.copy()
      for i2 in range(len(Y_train_i)):
        Y_train_i[i2]=1 if Y_train_i[i2]==i else 0
      j,w=gradient_descent(X_train,Y_train_i,w_ori.copy(),alpha[a],epoch[a],0)
      predictions.append(pred_fn(X_train,w))
    correct=0
    Y_pred=np.empty((len(Y_test),1), dtype=int)
    for i in range(len(Y_test)):
      prediction=np.zeros(shape=3)
      prediction[0] = predictions[0][i]
      prediction[1] = predictions[1][i]
      prediction[2] = predictions[2][i]
      maxp=max(prediction)
      best, = np.where(np.isclose(prediction, maxp))
      Y_pred[i]= best[0]
      if Y_pred[i]==Y_test[i]:
        correct+=1
    accuracy_fold[fold]=100.0*correct/len(Y_test)

    for i in range(3):
      p_pred=0
      p_pred_correct=0
      p_actual=0
      for x in range(len(Y_test)):
        if Y_pred[x]==i:
          p_pred+=1
          if Y_pred[x]==Y_test[x]:
            p_pred_correct+=1
        if Y_test[x]==i:
          p_actual+=1
      class_precision[fold][i]=0.0 if p_pred==0 else p_pred_correct*1.0/(p_pred*1.0)
      class_recall[fold][i]=0.0 if p_actual ==0 else p_pred_correct*1.0/(p_actual*1.0)
      print(i,"th class",p_pred,p_pred_correct,p_actual)
    fold+=1
'''print(accuracy_fold)
print(class_precision)
print(class_recall)'''
print("Class wise stats")
for i in range(3):
  print("Class ",i)
  print("Precision ",np.average(class_precision[:,i]))
  print("Recall ",np.average(class_recall[:,i]))
print("Fold wise accuracy")
for i in range(k):
  print(i,"th fold ",accuracy_fold[i])
print("Overall accuracy ",np.average(accuracy_fold))

0 th fold...
0 th class 0 0 20
1 th class 57 25 50
2 th class 63 32 50
1 th fold...
0 th class 6 4 30
1 th class 30 15 40
2 th class 84 41 50
2 th fold...
0 th class 56 29 50
1 th class 0 0 20
2 th class 64 31 50
3 th fold...
0 th class 80 37 50
1 th class 29 9 40
2 th class 11 5 30
4 th fold...
0 th class 60 28 50
1 th class 60 29 50
2 th class 0 0 20
Class wise stats
Class  0
Precision  0.4227380952380953
Recall  0.4026666666666666
Class  1
Precision  0.3464549304295221
Recall  0.336
Class  2
Precision  0.38699044011544015
Recall  0.4493333333333333
Fold wise accuracy
0 th fold  [47.5]
1 th fold  [50.]
2 th fold  [50.]
3 th fold  [42.5]
4 th fold  [47.5]
Overall accuracy  47.5
